# New section

checking if the gpu is avaliable 

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat May  6 12:39:46 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    50W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os

# Print the current working directory
print("Current working directory: {0}".format(os.getcwd()))

Current working directory: /content


In [ ]:
import numpy as np 
df = np.load('/content/drive/MyDrive/Colab Notebooks/3k_dic.npy')

In [ ]:
#loading in the labels df 
import pandas as pd 
label = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Labels.csv")

In [ ]:
#one hot encoding into one column 
label.loc[label['Normal'] == 1, 'one_hot_encoded_col'] = 0
label.loc[label['Actionable'] == 1, 'one_hot_encoded_col'] = 1
label.loc[label['Benign'] == 1, 'one_hot_encoded_col'] = 2
label.loc[label['Cancer'] == 1, 'one_hot_encoded_col'] = 3

# display the resulting dataframe
print(label)

       PatientID    StudyUID  View  Normal  Actionable  Benign  Cancer  \
0     DBT-P00023  DBT-S04378  lmlo       0           1       0       0   
1     DBT-P00023  DBT-S04378  rmlo       0           1       0       0   
2     DBT-P00029  DBT-S00044  lmlo       1           0       0       0   
3     DBT-P00029  DBT-S00044  rmlo       1           0       0       0   
4     DBT-P00032  DBT-S04706  lmlo       1           0       0       0   
...          ...         ...   ...     ...         ...     ...     ...   
9483  DBT-P02569  DBT-S01802  rmlo       1           0       0       0   
9484  DBT-P04356  DBT-S00073  lmlo       1           0       0       0   
9485  DBT-P04356  DBT-S00073  rmlo       1           0       0       0   
9486  DBT-P01481  DBT-S01753  lmlo       1           0       0       0   
9487  DBT-P01481  DBT-S01753  rmlo       1           0       0       0   

      one_hot_encoded_col  
0                     1.0  
1                     1.0  
2                     0.0  

In [ ]:
#making list of first 3000 observations 
one_hot_encoded_list = label.iloc[:3000]['one_hot_encoded_col'].tolist()


In [ ]:
#making list of all observations 
one_hot_encoded_list = label['one_hot_encoded_col'].tolist()



In [ ]:
from collections import Counter

one_hot_encoded_counts = Counter(one_hot_encoded_list)
print(one_hot_encoded_counts)


Counter({0.0: 9116, 1.0: 358, 2.0: 10, 3.0: 4})


This is a cnn model with no batch processes 

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.resnet50 import ResNet50

if tf.test.is_gpu_available():
    print('GPU available:')
    # Print the name of the GPU(s) available
    for gpu in tf.config.list_physical_devices('GPU'):
        print(gpu)
    # Set TensorFlow to use the first GPU
    tf.config.set_visible_devices(gpu, 'GPU')
else:
    print('No GPU available.')

# Load the DICOM array (assuming it has shape [num_images, depth, height, width, num_channels])
dicom_array = df
labels = one_hot_encoded_list

# Repeat single channel 3 times to create 3-channel input
dicom_array = np.repeat(dicom_array, 3, axis=-1)

# Convert labels to one-hot encoded vectors
labels = np.asarray(labels)
labels = labels.astype(np.int32)
num_classes = np.max(labels) + 1  # Add 1 to account for 0-indexing
labels_one_hot = tf.keras.utils.to_categorical(labels, num_classes)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    dicom_array, labels_one_hot, test_size=0.2, random_state=42, stratify=labels
)

# Parameters
input_shape = (5, 280, 280, 3)

# Load the pre-trained 2D ResNet50 model, excluding the top layer
base_model = ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=input_shape[1:]
)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Custom model to handle the 5D input
input_layer = layers.Input(shape=input_shape, name="input")
x = layers.TimeDistributed(base_model)(input_layer)
x = layers.Conv3D(64, kernel_size=(1, 3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling3D(pool_size=(1, 2, 2))(x)
x = layers.BatchNormalization()(x)
x = layers.GlobalAveragePooling3D()(x)
x = layers.Dropout(0.3)(x)
output_layer = layers.Dense(num_classes, activation='softmax', name="output")(x)

# Create the model
model = keras.Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print the model summary
model.summary()

# Train the model with your data
history = model.fit(X_train, y_train, epochs=10, batch_size= 4 , validation_split=0.2)




Attempting CNN withbatch processes 

In [ ]:
score = model.evaluate(X_test, y_test, verbose=1)
print("Test loss:", score[0])
print("Test accuracy:", score[1])


In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score, confusion_matrix

# Predict the output of the model on the test set
y_pred = model.predict(X_test)

# Calculate ROC AUC score
auc = roc_auc_score(y_test, y_pred, multi_class='ovr')

# Calculate confusion matrix
y_true_classes = np.argmax(y_test, axis=1)
y_pred_classes = np.argmax(y_pred, axis=1)
cm = confusion_matrix(y_true_classes, y_pred_classes)

# Calculate sensitivity and specificity for each class
num_classes = y_test.shape[1]
sensitivity = []
specificity = []
for i in range(num_classes):
    true_positives = cm[i,i]
    false_positives = cm[:,i].sum() - true_positives
    false_negatives = cm[i,:].sum() - true_positives
    true_negatives = cm.sum() - true_positives - false_positives - false_negatives
    
    sensitivity.append(true_positives / (true_positives + false_negatives))
    specificity.append(true_negatives / (true_negatives + false_positives))

print('ROC AUC score:', auc)
print('Sensitivity:', sensitivity)
print('Specificity:', specificity)

19/19 [==============================] - 9s 210ms/step
ROC AUC score: 0.5971781851654286
Sensitivity: [0.9980952380952381, 0.0, 0.0, 0.0]
Specificity: [0.0, 0.9981060606060606, 1.0, 1.0]




```
`
```

